# Notebook Imports

In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV

import statistics
from statistics import mode

import joblib

import time

# Writing a Pick-Suggesting Function

## Second Pick - Phase 1 Function

In [2]:
df_reshaped = pd.read_pickle('./saved_files/data_final.pickle')

In [3]:
# Adding variables for the function
# defining features and targets
# selecting features
sp_1p_features = df_reshaped[['left_pick_order_1', 'left_pick_order_2', 
                              'right_pick_order_1',
                              'left_attack_damage_1', 'left_attack_damage_2',
                              'left_receive_damage_1', 'left_receive_damage_2',
                              'left_kill_count_1', 'left_kill_count_2',
                              'right_attack_damage_1',
                              'right_receive_damage_1', 
                              'right_kill_count_1',
                              'left_attribute_cd_1', 'left_attribute_cd_2', 
                              'left_job_cd_1', 'left_job_cd_2',
                              'right_hero_code_1', 
                              'right_attribute_cd_1', 
                              'right_job_cd_1', 
                              'left_preban_1', 'left_preban_2', 
                              'right_preban_1', 'right_preban_2',
                              'first_pick', 'is_win']]

sp_1p_target = df_reshaped[['left_hero_code_1', 'left_hero_code_2']]

X_train_sp_1p, X_test_sp_1p, y_train_sp_1p, y_test_sp_1p = train_test_split(sp_1p_features, sp_1p_target, test_size= 0.2, shuffle = True, random_state= 890)

# Scaling 
# Define numeric labels
sp_1p_numeric = ['left_attack_damage_1', 'left_attack_damage_2',
                 'left_receive_damage_1', 'left_receive_damage_2', 
                 'left_kill_count_1', 'left_kill_count_2',
                 'right_attack_damage_1',
                 'right_receive_damage_1', 
                 'right_kill_count_1']

# Scale numeric features
sp_1p_scaler = StandardScaler()
sp_1p_scaler.fit(X_train_sp_1p[sp_1p_numeric])
X_train_sp_1p_trans = pd.DataFrame()
X_test_sp_1p_trans = pd.DataFrame()

X_train_sp_1p_trans[sp_1p_numeric] = sp_1p_scaler.transform(X_train_sp_1p[sp_1p_numeric])
X_test_sp_1p_trans[sp_1p_numeric] = sp_1p_scaler.transform(X_test_sp_1p[sp_1p_numeric])

# Encoding the categorical features
# Initializing an ordinal encoder for categorical variables
sp_1p_enc = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

# categorical labels for the 1st iter
sp_1p_categorical = ['left_attribute_cd_1', 'left_attribute_cd_2',
                     'left_job_cd_1', 'left_job_cd_2',
                     'right_hero_code_1',
                     'right_attribute_cd_1',
                     'right_job_cd_1', 
                     'left_preban_1', 'left_preban_2',
                     'right_preban_1', 'right_preban_2']

X_train_sp_1p_trans[sp_1p_categorical] = sp_1p_enc.fit_transform(X_train_sp_1p[sp_1p_categorical].to_numpy())
X_test_sp_1p_trans[sp_1p_categorical] = sp_1p_enc.transform(X_test_sp_1p[sp_1p_categorical].to_numpy())
X_train_sp_1p_trans.index
X_test_sp_1p_trans.index
X_train_sp_1p.index
X_test_sp_1p.index
X_test_sp_1p.isna().sum()
X_test_sp_1p_trans.isna().sum()
X_test_sp_1p_trans.shape
X_test_sp_1p.shape

# add features left out of scaling
l0f_sp_1p = ['left_pick_order_1', 'left_pick_order_2',
             'right_pick_order_1', 
             'first_pick', 'is_win']

X_train_sp_1p_trans[l0f_sp_1p] = X_train_sp_1p[l0f_sp_1p]
X_test_sp_1p_trans[l0f_sp_1p] = X_test_sp_1p[l0f_sp_1p]

# Loading df with hero codes and names
dict_sp_1p = pd.read_pickle('./saved_files/dict_sp_1p.pickle')

# Loading the model
classifier_sp_1p = joblib.load('./saved_files/classifier_sp_1p.joblib')

In [4]:
sp_1p_features

,left_pick_order_1,left_pick_order_2,right_pick_order_1,left_attack_damage_1,left_attack_damage_2,left_receive_damage_1,left_receive_damage_2,left_kill_count_1,left_kill_count_2,right_attack_damage_1,...,left_job_cd_2,right_hero_code_1,right_attribute_cd_1,right_job_cd_1,left_preban_1,left_preban_2,right_preban_1,right_preban_2,first_pick,is_win
0,1,2,1,0,1267,0,29297,0,0,0,...,manauser,Death Dealer Ray,dark,manauser,Zio,New Moon Luna,Sea Phantom Politis,New Moon Luna,1,1
1,1,2,1,3112,0,0,0,0,0,11226,...,knight,Dragon Bride Senya,light,knight,Zio,New Moon Luna,Blood Moon Haste,Jenua,0,1
2,1,2,1,0,4084,0,93232,0,1,0,...,manauser,Ambitious Tywin,light,knight,Zio,New Moon Luna,Ran,New Moon Luna,1,1
3,1,2,1,2043,16469,51842,19373,0,1,6272,...,manauser,Death Dealer Ray,dark,manauser,Zio,New Moon Luna,Sea Phantom Politis,Peira,1,1
4,1,2,1,0,0,0,0,0,0,0,...,knight,Death Dealer Ray,dark,manauser,New Moon Luna,Zio,Nahkwol,Sea Phantom Politis,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9941,1,2,1,23619,0,74779,0,1,0,65588,...,knight,Urban Shadow Choux,dark,warrior,Sea Phantom Politis,Jenua,Ran,New Moon Luna,0,2
9942,1,2,1,3235,0,17022,75373,0,0,13391,...,warrior,Ambitious Tywin,light,knight,Sea Phantom Politis,Jenua,Sea Phantom Politis,Jenua,0,2
9943,1,2,1,497,3219,34720,46092,0,0,15913,...,warrior,Blood Moon Haste,dark,manauser,Sea Phantom Politis,Jenua,Jenua,New Moon Luna,1,1
9944,1,2,1,11488,38100,85987,126078,1,3,31899,...,knight,Death Dealer Ray,dark,manauser,Sea Phantom Politis,Jenua,Navy Captain Landy,Belian,0,2


In [5]:
def sp_1_2(enemy_pick_1,
           left_preban_1,
           left_preban_2,
           right_preban_1,
           right_preban_2):
    """    
    First and second picks suggestion

    - Parameters: prebans, 1st hero picked by the opponent
    
    Returns:
    - array: hero names
    """
    
    test_features_1_2 = pd.DataFrame(columns = X_train_sp_1p_trans.columns)
    # configure stats
    #prebans
    test_features_1_2.loc[0, 'left_preban_1'] = dict_sp_1p[dict_sp_1p['left_preban_1_orig'] == left_preban_1]['left_preban_1'].values[0] # left_preban_1
    test_features_1_2.loc[0, 'left_preban_2'] = dict_sp_1p[dict_sp_1p['left_preban_2_orig'] == left_preban_2]['left_preban_2'].values[0] # left_preban_2 
    test_features_1_2.loc[0, 'right_preban_1'] = dict_sp_1p[dict_sp_1p['right_preban_1_orig'] == right_preban_1]['right_preban_1'].values[0] # right_preban_1 
    test_features_1_2.loc[0, 'right_preban_2'] = dict_sp_1p[dict_sp_1p['right_preban_2_orig'] == right_preban_2]['right_preban_2'].values[0] # right_preban_2

    # right side picks
    test_features_1_2.loc[0, 'right_attribute_cd_1'] = dict_sp_1p[dict_sp_1p['right_hero_code_1_orig'] == enemy_pick_1]['right_attribute_cd_1'].values[0] # right_attribute_cd_1
    test_features_1_2.loc[0, 'right_job_cd_1'] = dict_sp_1p[dict_sp_1p['right_hero_code_1_orig'] == enemy_pick_1]['right_job_cd_1'].values[0] # right_job_cd_1
    test_features_1_2.loc[0, 'right_hero_code_1'] = dict_sp_1p[dict_sp_1p['right_hero_code_1_orig'] == enemy_pick_1]['right_hero_code_1'].values[0] # right_hero_code_1 

    # supply mean numeric values as default encoded numeric features
    battle_stats = sp_1p_features[sp_1p_numeric].astype(float).mean().values.reshape(1,9)
    for i in range(len(test_features_1_2[sp_1p_numeric].columns)):
        test_features_1_2[sp_1p_numeric[i]] = battle_stats[0][i]
    
    # supply unencoded numeric values as default numeric features
    test_features_1_2['left_pick_order_1'] = 1 
    test_features_1_2['left_pick_order_2'] = 2
    test_features_1_2['right_pick_order_1'] = 1
    test_features_1_2['first_pick'] = 0
    test_features_1_2['is_win'] = 1
    
              
    # make prediction
    hero_names_1_2 = classifier_sp_1p.predict(test_features_1_2)
    print('1st & 2nd pick:', hero_names_1_2)
    return hero_names_1_2, enemy_pick_1

In [6]:
# Test the function
sp_1_2(enemy_pick_1 = 'Blood Moon Haste',
       left_preban_1 = 'Laia',
       left_preban_2 = 'Death Dealer Ray',
       right_preban_1 = 'Jenua',
       right_preban_2 = 'New Moon Luna')

1st & 2nd pick: [['Dragon Bride Senya' 'Dragon Bride Senya']]


(array([['Dragon Bride Senya', 'Dragon Bride Senya']], dtype=object),
 'Blood Moon Haste')

## Second Pick - Phase 2 Function

In [7]:
# Adding variables for the function
# defining features and targets
# select features

# selected features for the 1st iter with Ordinal Encoding
sp_2p_features = df_reshaped[['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3', 'left_pick_order_4', 
                              'right_pick_order_1', 'right_pick_order_2', 'right_pick_order_3', 
                              'left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3', 'left_attack_damage_4', 
                              'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3', 'left_receive_damage_4', 
                              'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3', 'left_kill_count_4',
                              'right_attack_damage_1', 'right_attack_damage_2', 'right_attack_damage_3',
                              'right_receive_damage_1', 'right_receive_damage_2', 'right_receive_damage_3',
                              'right_kill_count_1', 'right_kill_count_2', 'right_kill_count_3',
                              'left_hero_code_1', 'left_hero_code_2',
                              'left_attribute_cd_1', 'left_attribute_cd_2', 'left_attribute_cd_3', 'left_attribute_cd_4',
                              'left_job_cd_1', 'left_job_cd_2', 'left_job_cd_3', 'left_job_cd_4',
                              'right_hero_code_1', 'right_hero_code_2', 'right_hero_code_3',
                              'right_attribute_cd_1', 'right_attribute_cd_2', 'right_attribute_cd_3',
                              'right_job_cd_1', 'right_job_cd_2', 'right_job_cd_3',
#                              'left_postban', 'right_postban',
                              'left_preban_1', 'left_preban_2', 'right_preban_1', 'right_preban_2', 
                              'first_pick', 'is_win']]

sp_2p_target = df_reshaped[['left_hero_code_3', 'left_hero_code_4']]

X_train_sp_2p, X_test_sp_2p, y_train_sp_2p, y_test_sp_2p = train_test_split(sp_2p_features, sp_2p_target, test_size= 0.2, shuffle = True, random_state= 890)

# Scaling 
# Define numeric labels
sp_2p_numeric = ['left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3', 'left_attack_damage_4',
                 'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3', 'left_receive_damage_4',
                 'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3', 'left_kill_count_4', 
                 'right_attack_damage_1', 'right_attack_damage_2', 'right_attack_damage_3',
                 'right_receive_damage_1', 'right_receive_damage_2', 'right_receive_damage_3',
                 'right_kill_count_1', 'right_kill_count_2', 'right_kill_count_3']

# Scale numeric features
sp_2p_scaler = StandardScaler()
sp_2p_scaler.fit(X_train_sp_2p[sp_2p_numeric])
X_train_sp_2p_trans = pd.DataFrame()
X_test_sp_2p_trans = pd.DataFrame()

X_train_sp_2p_trans[sp_2p_numeric] = sp_2p_scaler.transform(X_train_sp_2p[sp_2p_numeric])
X_test_sp_2p_trans[sp_2p_numeric] = sp_2p_scaler.transform(X_test_sp_2p[sp_2p_numeric])

# Encoding the categorical features
# Initializing an ordinal encoder for categorical variables
sp_2p_enc = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

# categorical features
sp_2p_categorical = ['left_hero_code_1', 'left_hero_code_2',
                     'left_preban_1', 'left_preban_2',
                     'right_preban_1', 'right_preban_2',
                     'left_attribute_cd_1', 'left_attribute_cd_2', 'left_attribute_cd_3', 'left_attribute_cd_4',
                     'left_job_cd_1', 'left_job_cd_2', 'left_job_cd_3', 'left_job_cd_4',
                     'right_hero_code_1', 'right_hero_code_2', 'right_hero_code_3', 
                     'right_attribute_cd_1', 'right_attribute_cd_2', 'right_attribute_cd_3',
                     'right_job_cd_1', 'right_job_cd_2', 'right_job_cd_3']

X_train_sp_2p_trans[sp_2p_categorical] = sp_2p_enc.fit_transform(X_train_sp_2p[sp_2p_categorical].to_numpy())
X_test_sp_2p_trans[sp_2p_categorical] = sp_2p_enc.transform(X_test_sp_2p[sp_2p_categorical].to_numpy())
X_train_sp_2p_trans.index
X_test_sp_2p_trans.index
X_train_sp_2p = X_train_sp_2p.reset_index(drop=True)
X_test_sp_2p = X_test_sp_2p.reset_index(drop=True)

# add features left out of scaling
l0f_sp_2p = ['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3', 'left_pick_order_4',
             'right_pick_order_1', 'right_pick_order_2', 'right_pick_order_3', 
             'first_pick', 'is_win']

X_train_sp_2p_trans[l0f_sp_2p] = X_train_sp_2p[l0f_sp_2p]
X_test_sp_2p_trans[l0f_sp_2p] = X_test_sp_2p[l0f_sp_2p]

# Loading df with hero codes and names
dict_sp_2p = pd.read_pickle('./saved_files/dict_sp_2p.pickle')

# Loading the model
classifier_sp_2p = joblib.load('./saved_files/classifier_sp_2p.joblib')

In [8]:
sp_2p_features

,left_pick_order_1,left_pick_order_2,left_pick_order_3,left_pick_order_4,right_pick_order_1,right_pick_order_2,right_pick_order_3,left_attack_damage_1,left_attack_damage_2,left_attack_damage_3,...,right_attribute_cd_3,right_job_cd_1,right_job_cd_2,right_job_cd_3,left_preban_1,left_preban_2,right_preban_1,right_preban_2,first_pick,is_win
0,1,2,3,4,1,2,3,0,1267,0,...,fire,manauser,knight,assassin,Zio,New Moon Luna,Sea Phantom Politis,New Moon Luna,1,1
1,1,2,3,4,1,2,3,3112,0,53383,...,wind,knight,warrior,assassin,Zio,New Moon Luna,Blood Moon Haste,Jenua,0,1
2,1,2,3,4,1,2,3,0,4084,6434,...,wind,knight,assassin,assassin,Zio,New Moon Luna,Ran,New Moon Luna,1,1
3,1,2,3,4,1,2,3,2043,16469,1228,...,dark,manauser,knight,warrior,Zio,New Moon Luna,Sea Phantom Politis,Peira,1,1
4,1,2,3,4,1,2,3,0,0,0,...,light,manauser,manauser,knight,New Moon Luna,Zio,Nahkwol,Sea Phantom Politis,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9941,1,2,3,4,1,2,3,23619,0,4764,...,ice,warrior,knight,manauser,Sea Phantom Politis,Jenua,Ran,New Moon Luna,0,2
9942,1,2,3,4,1,2,3,3235,0,0,...,light,knight,assassin,warrior,Sea Phantom Politis,Jenua,Sea Phantom Politis,Jenua,0,2
9943,1,2,3,4,1,2,3,497,3219,12001,...,light,manauser,assassin,manauser,Sea Phantom Politis,Jenua,Jenua,New Moon Luna,1,1
9944,1,2,3,4,1,2,3,11488,38100,0,...,dark,manauser,mage,assassin,Sea Phantom Politis,Jenua,Navy Captain Landy,Belian,0,2


In [9]:
# Get mean value of encoded numeric features 
battle_stats = sp_2p_features[sp_2p_numeric].astype(float).mean().values.reshape(1,21)
battle_stats.shape

(1, 21)

In [10]:
battle_stats[0][0]

8358.296401132227

In [11]:
def sp_3_4(enemy_pick_1,
           enemy_pick_2,
           enemy_pick_3,
           my_sp1,
           my_sp2,
           left_preban_1,
           left_preban_2,
           right_preban_1,
           right_preban_2):
    """    
    First and second picks suggestion

    - Parameters: prebans, 1st, 2nd and 3rd heroes picked by the opponent
    
    Returns:
    - array: hero names
    """
    
    test_features_3_4 = pd.DataFrame(columns = X_train_sp_2p_trans.columns)
    
    # configure stats
    #prebans
    test_features_3_4.loc[0, 'left_preban_1'] = dict_sp_2p[dict_sp_2p['left_preban_1_orig'] == left_preban_1]['left_preban_1'].values[0] # left_preban_1
    test_features_3_4.loc[0, 'left_preban_2'] = dict_sp_2p[dict_sp_2p['left_preban_2_orig'] == left_preban_2]['left_preban_2'].values[0] # left_preban_2 
    test_features_3_4.loc[0, 'right_preban_1'] = dict_sp_2p[dict_sp_2p['right_preban_1_orig'] == right_preban_1]['right_preban_1'].values[0] # right_preban_1 
    test_features_3_4.loc[0, 'right_preban_2'] = dict_sp_2p[dict_sp_2p['right_preban_2_orig'] == right_preban_2]['right_preban_2'].values[0] # right_preban_2

    # right side picks
    test_features_3_4.loc[0, 'right_attribute_cd_1'] = dict_sp_2p[dict_sp_2p['right_hero_code_1_orig'] == enemy_pick_1]['right_attribute_cd_1'].values[0] # right_attribute_cd_1
    test_features_3_4.loc[0, 'right_job_cd_1'] = dict_sp_2p[dict_sp_2p['right_hero_code_1_orig'] == enemy_pick_1]['right_job_cd_1'].values[0] # right_job_cd_1
    test_features_3_4.loc[0, 'right_hero_code_1'] = dict_sp_2p[dict_sp_2p['right_hero_code_1_orig'] == enemy_pick_1]['right_hero_code_1'].values[0] # right_hero_code_1 
    test_features_3_4.loc[0, 'right_attribute_cd_2'] = dict_sp_2p[dict_sp_2p['right_hero_code_2_orig'] == enemy_pick_2]['right_attribute_cd_2'].values[0] # right_attribute_cd_2
    test_features_3_4.loc[0, 'right_job_cd_2'] = dict_sp_2p[dict_sp_2p['right_hero_code_2_orig'] == enemy_pick_2]['right_job_cd_2'].values[0] # right_job_cd_2
    test_features_3_4.loc[0, 'right_hero_code_2'] = dict_sp_2p[dict_sp_2p['right_hero_code_2_orig'] == enemy_pick_2]['right_hero_code_2'].values[0] # right_hero_code_2 
    test_features_3_4.loc[0, 'right_attribute_cd_3'] = dict_sp_2p[dict_sp_2p['right_hero_code_3_orig'] == enemy_pick_3]['right_attribute_cd_3'].values[0] # right_attribute_cd_3
    test_features_3_4.loc[0, 'right_job_cd_3'] = dict_sp_2p[dict_sp_2p['right_hero_code_3_orig'] == enemy_pick_3]['right_job_cd_3'].values[0] # right_job_cd_3
    test_features_3_4.loc[0, 'right_hero_code_3'] = dict_sp_2p[dict_sp_2p['right_hero_code_3_orig'] == enemy_pick_3]['right_hero_code_3'].values[0] # right_hero_code_3

    # left side picks
    test_features_3_4.loc[0, 'left_attribute_cd_1'] = dict_sp_2p[dict_sp_2p['left_hero_code_1_orig'] == my_sp1]['left_attribute_cd_1'].values[0] # left_attribute_cd_1
    test_features_3_4.loc[0, 'left_attribute_cd_2'] = dict_sp_2p[dict_sp_2p['left_hero_code_2_orig'] == my_sp2]['left_attribute_cd_2'].values[0] # left_attribute_cd_2
    test_features_3_4.loc[0, 'left_job_cd_1'] = dict_sp_2p[dict_sp_2p['left_hero_code_1_orig'] == my_sp1]['left_job_cd_1'].values[0] # left_job_cd_1
    test_features_3_4.loc[0, 'left_job_cd_2'] = dict_sp_2p[dict_sp_2p['left_hero_code_2_orig'] == my_sp2]['left_job_cd_2'].values[0] # left_job_cd_2
    test_features_3_4.loc[0, 'left_hero_code_1'] = dict_sp_2p[dict_sp_2p['left_hero_code_1_orig'] == my_sp1]['left_hero_code_1'].values[0] # left_hero_code_1
    test_features_3_4.loc[0, 'left_hero_code_2'] = dict_sp_2p[dict_sp_2p['left_hero_code_2_orig'] == my_sp2]['left_hero_code_2'].values[0] # left_hero_code_2
    
    # supply mean numeric values as default encoded numeric features
    battle_stats = sp_2p_features[sp_2p_numeric].astype(float).mean().values.reshape(1,21)
    for i in range(len(test_features_3_4[sp_2p_numeric].columns)):
        test_features_3_4[sp_2p_numeric[i]] = battle_stats[0][i]
    
    # supply unencoded numeric values as default numeric features
    test_features_3_4['left_pick_order_1'] = 1 
    test_features_3_4['left_pick_order_2'] = 2
    test_features_3_4['left_pick_order_2'] = 3
    test_features_3_4['left_pick_order_2'] = 4
    test_features_3_4['right_pick_order_1'] = 1
    test_features_3_4['right_pick_order_1'] = 2
    test_features_3_4['right_pick_order_1'] = 3
    test_features_3_4['first_pick'] = 0
    test_features_3_4['is_win'] = 1
    
              
    # make prediction
    hero_names_3_4 = classifier_sp_2p.predict(test_features_3_4)
    print('3rd & 4th pick:', hero_names_3_4)
    return hero_names_3_4

In [18]:
# Test the function
sp_3_4(enemy_pick_1 = 'Blood Moon Haste',
       enemy_pick_2 = 'Ocean Breeze Luluca',
       enemy_pick_3 = 'Nahkwol',
       my_sp1 = 'Dragon Bride Senya',
       my_sp2 = 'Lua',
       left_preban_1 = 'Laia',
       left_preban_2 = 'Death Dealer Ray',
       right_preban_1 = 'Jenua',
       right_preban_2 = 'New Moon Luna')

3rd & 4th pick: [['Belian' 'Sylvan Sage Vivian']]


array([['Belian', 'Sylvan Sage Vivian']], dtype=object)

## Second Pick - Phase 3 Function

In [14]:
# Adding variables for the function

# Defining features and targets

# Select features
sp_3p_features = df_reshaped[['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3', 'left_pick_order_4', 'left_pick_order_5',
                              'right_pick_order_1', 'right_pick_order_2', 'right_pick_order_3', 'right_pick_order_4', 'right_pick_order_5',
                              'left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3', 'left_attack_damage_4', 'left_attack_damage_5', 
                              'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3', 'left_receive_damage_4', 'left_receive_damage_5',
                              'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3', 'left_kill_count_4', 'left_kill_count_5',
                              'right_attack_damage_1', 'right_attack_damage_2', 'right_attack_damage_3','right_attack_damage_4', 'right_attack_damage_5',
                              'right_receive_damage_1', 'right_receive_damage_2', 'right_receive_damage_3', 'right_receive_damage_4', 'right_receive_damage_5',
                              'right_kill_count_1', 'right_kill_count_2', 'right_kill_count_3', 'right_kill_count_4', 'right_kill_count_5',
                              'left_hero_code_1', 'left_hero_code_2', 'left_hero_code_3', 'left_hero_code_4', 
                              'left_preban_1', 'left_preban_2',
                              'right_preban_1', 'right_preban_2',
                              'left_attribute_cd_1', 'left_attribute_cd_2', 'left_attribute_cd_3', 'left_attribute_cd_4', 'left_attribute_cd_5',
                              'left_job_cd_1', 'left_job_cd_2', 'left_job_cd_3', 'left_job_cd_4', 'left_job_cd_5',
                              'right_hero_code_1', 'right_hero_code_2', 'right_hero_code_3', 'right_hero_code_4', 'right_hero_code_5',
                              'right_attribute_cd_1', 'right_attribute_cd_2', 'right_attribute_cd_3','right_attribute_cd_4', 'right_attribute_cd_5',
                              'right_job_cd_1', 'right_job_cd_2', 'right_job_cd_3', 'right_job_cd_4', 'right_job_cd_5',
                              'first_pick', 'is_win']]

sp_3p_target = df_reshaped[['left_hero_code_5']]

X_train_sp_3p, X_test_sp_3p, y_train_sp_3p, y_test_sp_3p = train_test_split(sp_3p_features, sp_3p_target, test_size= 0.2, shuffle = True, random_state= 890)



# Scaling 
# Define numeric labels
sp_3p_numeric = ['left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3', 'left_attack_damage_4', 'left_attack_damage_5', 
                 'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3', 'left_receive_damage_4', 'left_receive_damage_5',
                 'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3', 'left_kill_count_4', 'left_kill_count_5', 
                 'right_attack_damage_1', 'right_attack_damage_2', 'right_attack_damage_3','right_attack_damage_4', 'right_attack_damage_5',
                 'right_receive_damage_1', 'right_receive_damage_2', 'right_receive_damage_3', 'right_receive_damage_4', 'right_receive_damage_5',
                 'right_kill_count_1', 'right_kill_count_2', 'right_kill_count_3', 'right_kill_count_4', 'right_kill_count_5']

# Scale numeric features
sp_3p_scaler = StandardScaler()
sp_3p_scaler.fit(X_train_sp_3p[sp_3p_numeric])
X_train_sp_3p_trans = pd.DataFrame()
X_test_sp_3p_trans = pd.DataFrame()
X_train_sp_3p_trans[sp_3p_numeric] = sp_3p_scaler.transform(X_train_sp_3p[sp_3p_numeric])
X_test_sp_3p_trans[sp_3p_numeric] = sp_3p_scaler.transform(X_test_sp_3p[sp_3p_numeric])

# Encoding the categorical features
# Initializing an ordinal encoder for categorical variables
sp_3p_enc = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

# categorical labels for the 1st iter
sp_3p_categorical = ['left_hero_code_1', 'left_hero_code_2', 'left_hero_code_3', 'left_hero_code_4',
                     'left_preban_1', 'left_preban_2',
                     'right_preban_1', 'right_preban_2',
                     'left_attribute_cd_1', 'left_attribute_cd_2', 'left_attribute_cd_3', 'left_attribute_cd_4', 'left_attribute_cd_5',
                     'left_job_cd_1', 'left_job_cd_2', 'left_job_cd_3', 'left_job_cd_4', 'left_job_cd_5',
                     'right_hero_code_1', 'right_hero_code_2', 'right_hero_code_3', 'right_hero_code_4', 'right_hero_code_5',
                     'right_attribute_cd_1', 'right_attribute_cd_2', 'right_attribute_cd_3','right_attribute_cd_4', 'right_attribute_cd_5', 
                     'right_job_cd_1', 'right_job_cd_2', 'right_job_cd_3', 'right_job_cd_4', 'right_job_cd_5']

X_train_sp_3p_trans[sp_3p_categorical] = sp_3p_enc.fit_transform(X_train_sp_3p[sp_3p_categorical].to_numpy())
X_test_sp_3p_trans[sp_3p_categorical] = sp_3p_enc.transform(X_test_sp_3p[sp_3p_categorical].to_numpy())

X_train_sp_3p = X_train_sp_3p.reset_index(drop=True)
X_test_sp_3p = X_test_sp_3p.reset_index(drop=True)

# add features left out of scaling
l0f_sp_3p = ['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3', 'left_pick_order_4', 'left_pick_order_5',
             'right_pick_order_1', 'right_pick_order_2', 'right_pick_order_3', 'right_pick_order_4', 'right_pick_order_5',
             'first_pick', 'is_win']

X_train_sp_3p_trans[l0f_sp_3p] = X_train_sp_3p[l0f_sp_3p]
X_test_sp_3p_trans[l0f_sp_3p] = X_test_sp_3p[l0f_sp_3p]

# Loading df with hero codes and names
dict_sp_3p = pd.read_pickle('./saved_files/dict_sp_3p.pickle')

# Loading the model
classifier_sp_3p = joblib.load('./saved_files/classifier_sp_3p.joblib')

In [16]:
battle_stats = sp_3p_features[sp_3p_numeric].astype(float).mean().values.reshape(1,30)
battle_stats.shape

(1, 30)

In [25]:
def sp_5(enemy_pick_1,
           enemy_pick_2,
           enemy_pick_3,
           enemy_pick_4,
           enemy_pick_5,
           my_sp1,
           my_sp2,
           my_sp3,
           my_sp4,
           left_preban_1,
           left_preban_2,
           right_preban_1,
           right_preban_2):
    
    """
    Fifth pick suggestion

    - Parameters: prebans, my heroes 1-4, heroes 1-5 picked by the opponent
    
    Returns:
    List of hero names
    """
    
    test_features_5 = pd.DataFrame(columns = X_train_sp_3p_trans.columns)
    # configure stats
    # prebans
    test_features_5.loc[0, 'left_preban_1'] = dict_sp_3p[dict_sp_3p['left_preban_1_orig'] == left_preban_1]['left_preban_1'].values[0] # left_preban_1
    test_features_5.loc[0, 'left_preban_2'] = dict_sp_3p[dict_sp_3p['left_preban_2_orig'] == left_preban_2]['left_preban_2'].values[0] # left_preban_2
    test_features_5.loc[0, 'right_preban_1'] = dict_sp_3p[dict_sp_3p['right_preban_1_orig'] == right_preban_1]['right_preban_1'].values[0] # right_preban_1
    test_features_5.loc[0, 'right_preban_2'] = dict_sp_3p[dict_sp_3p['right_preban_2_orig'] == right_preban_2]['right_preban_2'].values[0] # right_preban_2
    
    # right side picks
    test_features_5.loc[0, 'right_attribute_cd_1'] = dict_sp_3p[dict_sp_3p['right_hero_code_1_orig'] == enemy_pick_1]['right_attribute_cd_1'].values[0] # right_attribute_cd_1
    test_features_5.loc[0, 'right_attribute_cd_2'] = dict_sp_3p[dict_sp_3p['right_hero_code_2_orig'] == enemy_pick_2]['right_attribute_cd_2'].values[0] # right_attribute_cd_2
    test_features_5.loc[0, 'right_attribute_cd_3'] = dict_sp_3p[dict_sp_3p['right_hero_code_3_orig'] == enemy_pick_3]['right_attribute_cd_3'].values[0] # right_attribute_cd_3
    test_features_5.loc[0, 'right_attribute_cd_4'] = dict_sp_3p[dict_sp_3p['right_hero_code_4_orig'] == enemy_pick_4]['right_attribute_cd_4'].values[0] # right_attribute_cd_4
    test_features_5.loc[0, 'right_attribute_cd_5'] = dict_sp_3p[dict_sp_3p['right_hero_code_5_orig'] == enemy_pick_5]['right_attribute_cd_5'].values[0] # right_attribute_cd_5
    test_features_5.loc[0, 'right_job_cd_1'] = dict_sp_3p[dict_sp_3p['right_hero_code_1_orig'] == enemy_pick_1]['right_job_cd_1'].values[0] # right_job_cd_1
    test_features_5.loc[0, 'right_job_cd_2'] = dict_sp_3p[dict_sp_3p['right_hero_code_2_orig'] == enemy_pick_2]['right_job_cd_2'].values[0] # right_job_cd_2
    test_features_5.loc[0, 'right_job_cd_3'] = dict_sp_3p[dict_sp_3p['right_hero_code_3_orig'] == enemy_pick_3]['right_job_cd_3'].values[0] # right_job_cd_3
    test_features_5.loc[0, 'right_job_cd_4'] = dict_sp_3p[dict_sp_3p['right_hero_code_4_orig'] == enemy_pick_4]['right_job_cd_4'].values[0] # right_job_cd_4
    test_features_5.loc[0, 'right_job_cd_5'] = dict_sp_3p[dict_sp_3p['right_hero_code_5_orig'] == enemy_pick_5]['right_job_cd_5'].values[0] # right_job_cd_5    
    test_features_5.loc[0, 'right_hero_code_1'] = dict_sp_3p[dict_sp_3p['right_hero_code_1_orig'] == enemy_pick_1]['right_hero_code_1'].values[0] # right_hero_code_1
    test_features_5.loc[0, 'right_hero_code_2'] = dict_sp_3p[dict_sp_3p['right_hero_code_2_orig'] == enemy_pick_2]['right_hero_code_2'].values[0] # right_hero_code_2
    test_features_5.loc[0, 'right_hero_code_3'] = dict_sp_3p[dict_sp_3p['right_hero_code_3_orig'] == enemy_pick_3]['right_hero_code_3'].values[0] # right_hero_code_3
    test_features_5.loc[0, 'right_hero_code_4'] = dict_sp_3p[dict_sp_3p['right_hero_code_4_orig'] == enemy_pick_4]['right_hero_code_4'].values[0] # right_hero_code_4
    test_features_5.loc[0, 'right_hero_code_5'] = dict_sp_3p[dict_sp_3p['right_hero_code_5_orig'] == enemy_pick_5]['right_hero_code_5'].values[0] # right_hero_code_5
    
    # left side picks
    test_features_5.loc[0, 'left_attribute_cd_1'] = dict_sp_3p[dict_sp_3p['left_hero_code_1_orig'] == my_sp1]['left_attribute_cd_1'].values[0] # left_attribute_cd_1
    test_features_5.loc[0, 'left_attribute_cd_2'] = dict_sp_3p[dict_sp_3p['left_hero_code_2_orig'] == my_sp2]['left_attribute_cd_2'].values[0] # left_attribute_cd_2
    test_features_5.loc[0, 'left_attribute_cd_3'] = dict_sp_3p[dict_sp_3p['left_hero_code_3_orig'] == my_sp3]['left_attribute_cd_3'].values[0] # left_attribute_cd_3
    test_features_5.loc[0, 'left_attribute_cd_4'] = dict_sp_3p[dict_sp_3p['left_hero_code_4_orig'] == my_sp4]['left_attribute_cd_4'].values[0] # left_attribute_cd_4
    test_features_5.loc[0, 'left_job_cd_1'] = dict_sp_3p[dict_sp_3p['left_hero_code_1_orig'] == my_sp1]['left_job_cd_1'].values[0] # left_job_cd_1
    test_features_5.loc[0, 'left_job_cd_2'] = dict_sp_3p[dict_sp_3p['left_hero_code_2_orig'] == my_sp2]['left_job_cd_2'].values[0] # left_job_cd_2
    test_features_5.loc[0, 'left_job_cd_3'] = dict_sp_3p[dict_sp_3p['left_hero_code_3_orig'] == my_sp3]['left_job_cd_3'].values[0] # left_job_cd_3
    test_features_5.loc[0, 'left_job_cd_4'] = dict_sp_3p[dict_sp_3p['left_hero_code_4_orig'] == my_sp4]['left_job_cd_4'].values[0] # left_job_cd_4
    test_features_5.loc[0, 'left_hero_code_1'] = dict_sp_3p[dict_sp_3p['left_hero_code_1_orig'] == my_sp1]['left_hero_code_1'].values[0] # left_hero_code_1
    test_features_5.loc[0, 'left_hero_code_2'] = dict_sp_3p[dict_sp_3p['left_hero_code_2_orig'] == my_sp2]['left_hero_code_2'].values[0] # left_hero_code_2
    test_features_5.loc[0, 'left_hero_code_3'] = dict_sp_3p[dict_sp_3p['left_hero_code_3_orig'] == my_sp3]['left_hero_code_3'].values[0] # left_hero_code_3
    test_features_5.loc[0, 'left_hero_code_4'] = dict_sp_3p[dict_sp_3p['left_hero_code_4_orig'] == my_sp3]['left_hero_code_4'].values[0] # left_hero_code_4
        
    
    # supply mean numeric values as default encoded numeric features
    battle_stats = sp_3p_features[sp_3p_numeric].astype(float).mean().values.reshape(1,30)
    for i in range(len(test_features_5[sp_3p_numeric].columns)):
        test_features_5[sp_3p_numeric[i]] = battle_stats[0][i]
    
    # supply unencoded numeric values as default numeric features
    test_features_5['left_pick_order_1'] = 1 
    test_features_5['left_pick_order_2'] = 2
    test_features_5['left_pick_order_3'] = 3
    test_features_5['left_pick_order_4'] = 4
    test_features_5['left_pick_order_5'] = 5
    test_features_5['right_pick_order_1'] = 1
    test_features_5['right_pick_order_2'] = 2
    test_features_5['right_pick_order_3'] = 3
    test_features_5['right_pick_order_4'] = 4
    test_features_5['right_pick_order_5'] = 5
    test_features_5['first_pick'] = 0
    test_features_5['is_win'] = 1
    
              
    # make prediction
    hero_names_5 = classifier_sp_3p.predict(test_features_5)
    print('4th & 5th pick:', hero_names_5)
    return hero_names_5

In [26]:
sp_5(enemy_pick_1 = 'Blood Moon Haste',
    enemy_pick_2 = 'Ocean Breeze Luluca',
    enemy_pick_3 = 'Nahkwol',
    enemy_pick_4 = 'Navy Captain Landy',
    enemy_pick_5 = 'Abyssal Yufine',
    my_sp1 = 'Dragon Bride Senya',
    my_sp2 = 'Lua',
    my_sp3 = 'Belian',
    my_sp4 = 'Sylvan Sage Vivian',
    left_preban_1 = 'Laia',
    left_preban_2 = 'Death Dealer Ray',
    right_preban_1 = 'Jenua',
    right_preban_2 = 'New Moon Luna')

4th & 5th pick: [['Sylvan Sage Vivian']]


array([['Sylvan Sage Vivian']], dtype=object)